In [ ]:
# need to upload the training corpus file named "small.wiki" 
# !wget https://drive.google.com/open?id=1EGKoNOlaR84UL6Y1luxUXuus3UX2tXQL

from google.colab import files
files.upload()

###### This jupyter sheet is an RPF generator. It can run on GC, Azure, AWS or Oracle Cloud.
For it to work, runtime has to be supported by GPU with 16GB or more (does not happen often on free tier)

In [1]:
# check your GPU luck
#import os
#os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit=/home/volkot/.local/lib/python3.6/site-packages/tensorflow/compiler/xla:$TF_XLA_FLAGS=--tf_xla_cpu_global_jit"
!echo $TF_XLA_FLAGS
!nvidia-smi
#

--tf_xla_cpu_global_jit=/home/volkot/.local/lib/python3.6/site-packages/tensorflow/compiler/xla:$TF_XLA_FLAGS=--tf_xla_cpu_global_jit


In [2]:
# @kernel-restart
# Random wiki article

input = """The cat (Felis catus) is a small carnivorous mammal. 
It is the only domesticated species in the family Felidae and often referred to as the domestic cat to distinguish it from wild members of the family.
The cat is either a house cat, a farm cat or a feral cat; latter ranges freely and avoids human contact. 
Domestic cats are valued by humans for companionship and for their ability to hunt rodents. About 60 cat breeds are recognized by various cat registries.
The cat is similar in anatomy to the other felid species, has a strong flexible body, quick reflexes, sharp teeth and retractable claws adapted to killing small prey. 
Its night vision and sense of smell are well developed. Cat communication includes vocalizations like meowing, purring, trilling, hissing, growling and grunting as well as cat-specific body language. 
It is a solitary hunter, but a social species. It can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small mammals. 
It is a predator that is most active at dawn and dusk.It secretes and perceives pheromones."""



In [8]:
# one-time
# install dependencies if not in ML environment

!pip install sacremoses
!pip install gpt-2-simple 
!pip install nltk
!pip install wrapt --upgrade --ignore-installed
!pip install --upgrade tensorflow-gpu==1.14
!pip install tensorflow-hub 

# instructions to enable 774M model:
#
# vi /home/ubuntu/anaconda3/lib/python3.7/site-packages/gpt_2_simple/gpt_2.py
# comment out line: assert model_name not in ['774M', '1558M']

Requirement already up-to-date: tensorflow-gpu==1.14 in ./anaconda3/lib/python3.7/site-packages (1.14.0)


In [3]:
# @kernel restart

import sys, getopt
import gc
from pathlib import Path
from sacremoses import MosesDetokenizer
import gpt_2_simple as gpt2
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd 
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

In [ ]:
# one-time
# download tokenizer file
nltk.download('punkt')

In [4]:
# @kernel restart
# GPT-2 released three models
#model_name="117M"
#model_name="774M"

model_name="345M"

# hyperparameters
minsimilarity = 0.88 # rejection threshold: 0.68 for lit.
# lexeme bounds
maxonepiece = 7      # max leftover chunk we are not willing to break
minwords = 3         # min lexeme length if it is a logical block ending in ;, -, --
softminwords = 7     # min lexeme ending in , -and-, or  
maxwords = 16        # max lexeme without a clear block end
# generator parameters
temperature = 0.92    # generator madness
seedLexemes = 10     # how many lexemes to put in a prior
nsamples = 10000      # number of samples generated to choose from


bestLexemes = []
output = ""

# this is your style for GPT-2 finetuning
style = "small.wiki"

# content input and output
generator_name = 'Trained-wiki'
outfile = generator_name+"_"+model_name+"_"+str(nsamples)+"_"+style
sfile = outfile+".stats"

# get around resource exhaustion for embeddings
config = tf.ConfigProto(
        device_count = {'GPU': 1} # may select 0 on Google Collab
        #device_count = {'GPU': 0}
    )
config.gpu_options.allow_growth = True


In [5]:
# @kernel restart
# init tokenizer and sentence embedder
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
dt = MosesDetokenizer()

# Google Sentence encoder v2 appears worse than V1
#module_url = "https://tfhub.dev/google/universal-sentence-encoder/1?tf-hub-format=compressed"
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"

embed = hub.Module(module_url)
similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_message_encodings = embed(similarity_input_placeholder)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
# download GPT-2 model (only needed once)
gpt2.download_gpt2(model_name=model_name)


In [6]:
# TRAINING
# set number of steps for finetuning or load the pretrained model
# if a model is already mounted, no training required

sess = gpt2.start_tf_sess()

# load an existing, fine-tuned model

# gpt2.load_gpt2(sess)
#
# finetuning frequently fails in collab based on resource exhaustion
# Good training duration results in avg loss between 0.2 and 0.9 (~20K training iterations on 345M model)
#
# GPT-2 can generate coherent output even on low finetuning, but with generic style (default)

# fine-tune from scratch

gpt2.finetune(sess, style, model_name=model_name, restore_from = 'latest', overwrite=True, steps=8000)

# continue training from a saved model if the kernel restarted

#gpt2.finetune(sess, style, model_name=model_name, overwrite=True, steps=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Loading checkpoint checkpoint/run1/model-8016
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from checkpoint/run1/model-8016


INFO:tensorflow:Restoring parameters from checkpoint/run1/model-8016
  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:17<00:00, 77.98s/it]


dataset has 12815132 tokens
Training...
Saving checkpoint/run1/model-8016
[8017 | 17.17] loss=2.78 avg=2.78
Saving checkpoint/run1/model-8017
Instructions for updating:
Use standard file APIs to delete files with this prefix.


Instructions for updating:
Use standard file APIs to delete files with this prefix.


In [7]:
# @every kernel restart
# take a list of unstructured expansions and tidy them up
# works around the following problem: the generated piece may include seed or not
#
# expansion formation rules: seed phrase deleted, truncation at first end 
# of sentence, the last comma, or allow to run off unclipped otherwise
# 

def cleaned(expansions, seed):

   seedbag = word_tokenize(seed)
   cutseed = dt.detokenize(seedbag[1:])
   symlen = len(cutseed)
   clean = []
   expansion = ""
   for exp in expansions:
      pos = exp.find(cutseed)
      if pos > 0:
         expansion = exp[pos+symlen:].strip()
      else:
         # sometimes search is thrown off w/special characters
         # heuristics here assumes expansion larger than seed must be trimmed
         expbag = word_tokenize(exp)
         if len(expbag) > len(seedbag):
            expansion = dt.detokenize(expbag[len(seedbag):])
         else:
            
         # all attempts to identify seed failed, assume generator gave no seed
            expansion = exp
            
      # try to end with a period
      pos = expansion.find(".")
      if pos > 0:
         expansion = expansion[:pos]+"."
      else: # or exclamation
         pos = expansion[::-1].find("!")
         if pos > 0:
            expansion = expansion[:-pos]
         else: # or comma
            pos = expansion[::-1].find(",")
            if pos > 0:
               expansion = expansion[:-pos]
      clean.append(expansion)
      
   return clean

In [ ]:
# test only
seed = """. he has an incredible appetite for her- -- and she, of course, did not take any interest in him at all! He has become quite a simpleminded man. And I think that, overall- Such an admirer of her performance on the stage as I was, and of her lyrical, and of her'northern music! "I can well remember," he said at one time listen to her for ever. Oh! and I had almost forgotten one idea that """
expansions = [""". he has an incredible appetite for her- -- and she, of course, did not take any interest in him at all! He has become quite a simpleminded man. And I think that, overall- Such an admirer of her performance on the stage as I was, and of her lyrical, and of her'northern music! "I can well remember," he said at one time listen to her for ever. Oh! and I had almost forgotten one idea that everything is fine"""]


In [ ]:
# test only
cleaned(expansions, seed)


In [ ]:
# test only
seedbag = word_tokenize(seed)
cutseed = dt.detokenize(seedbag[1:])
symlen = len(cutseed)
cleaned = []
expansion = ""
pos = expansions[0].find(cutseed)

In [ ]:
# test only
generateLexeme("")

In [8]:
# @every kernel restart
# define the next piece to replace in the donor text
def generateLexeme(sent):

      if sent == "":
         return

      words = word_tokenize(sent)

      lexeme = ""
      count = 0

      if len(words) <= maxonepiece:
         processLexeme(sent) # too small to split

      else:
         for word in words:
            count += 1
            if (word in [";","-","--"] and count > minwords) or (word in [",", "and", ":", "for", "at"] and count > softminwords) or count > maxwords:
               if (len(words)-count) >  minwords:
                  processLexeme(dt.detokenize(words[:count]))
                  sentence = dt.detokenize(words[count:])
                  generateLexeme(sentence)
               else:
                  processLexeme(dt.detokenize(words))
               break
         if count == len(words): # sentence could not be split in any reasonable way, give up.
            processLexeme(sent)
            

In [9]:
# @every kernel restart
# main RPS routine: extract a lexeme, generate candidates, replace with a best one
def processLexeme(sent):

   # always start from the first phrase
   if not bestLexemes:
      bestLexemes.append(sent)
      print("Leading output with seed phrase: {}".format(sent))
      return
    
   seed = dt.detokenize(bestLexemes[-seedLexemes:])
   print("seed: \"{}\"".format(seed))
   print("\t  processing lexeme: \"{}\"".format(sent))
   
   expansions = gpt2.generate(sess, nsamples=nsamples, batch_size=10, length=20, prefix=seed, return_as_list=True, include_prefix=False)
   #expansions = gpt2.generate(sess, nsamples=nsamples, batch_size=10, length=20, prefix=seed, truncate=".", return_as_list=True, include_prefix=False)
   #expansions += gpt2.generate(sess, nsamples=nsamples, batch_size=10, length=20, prefix=seed, truncate=",", return_as_list=True, include_prefix=False)
   seeds_removed = cleaned(expansions, seed)
   #expansions = [tokenizer.tokenize(expansion)[0] for expansion in seeds_removed]
   expansions = [sent_tokenize(expansion)[0] for expansion in seeds_removed]
   #print("  expansion set: {}".format(expansions))
  
   messages = [sent]+expansions
   bestLexeme = sent
  
   gc.collect()

   with tf.Session(config=config) as session:
      session.run(tf.global_variables_initializer())
      session.run(tf.tables_initializer())
      message_embeddings_ = session.run(similarity_message_encodings, feed_dict={similarity_input_placeholder: messages})

      corr = np.inner(message_embeddings_, message_embeddings_)
      embeddings = corr[0,1:]
      #print("embeddings: {}".format(embeddings))
      
      bestIndex = np.argmax(embeddings)
      
      if embeddings[bestIndex] >  minsimilarity:
        
         bestLexeme = expansions[bestIndex]
            
         # stats       
         # calculate running average for accepted scores

         acceptedLexemes["accepted"] = acceptedLexemes["accepted"]+1
         acceptedLexemes["candidate_percent"].append(np.sum(embeddings > minsimilarity)/nsamples)
            
         # join expansion with future lexeme smoothly
        
         originalEnding = word_tokenize(sent)[-1]
         originalEnding = " " + originalEnding if originalEnding in ["and","for", "at"] else originalEnding
  
         print("Original ending: {}".format(originalEnding))
         end = originalEnding if originalEnding in [".",",", ";", ":", "-", "--", "?", "!", "...", " and"," for", " at"] else ""

        
         lexemeEnding = word_tokenize(bestLexeme)[-1]
         
         # replace punctuation at end of lexeme if original punctuation existed
         # if donor sentence not ending yet, ignore the expansion markers
         if lexemeEnding in [".", ",", ";"]:
         #if lexemeEnding in [".", ",", ";", "?", "!"]:
            if end in [".", "?", "!"]:
               bestLexeme = bestLexeme[:-1]+end
            
            # bestLexeme = bestLexeme # do nothing 
         # else plug from the content donor. Removing this line allows a lot of freedom to style generator
         else:
            bestLexeme = bestLexeme+end
            
      else:
         # now need to harmonize with the end of the prior lexeme (if exists)

         if bestLexemes:
            lastLexeme = bestLexemes[-1]
            if lastLexeme:
                lastLexemeEnd = lastLexeme[-1]
                # if the best lexeme starts a new sentence, force-end the last lexeme
                bestLexemeStart = bestLexeme[0]
                if bestLexemeStart.isupper() and lastLexemeEnd not in ['.', '!','?','...']:
                    if lastLexemeEnd in [',', ';','-','--']: 
                        lastLexeme = lastLexemeEnd[:-1]
                    lastLexeme=lastLexeme+"."
                    bestLexemes[-1]=lastLexeme
                # if last lexeme ended in a terminator, start from a capital letter
                lastLexemeEnd = lastLexeme[-1]
                if bestLexemeStart.islower() and lastLexemeEnd in ['.', '!','?','...']:
                    #lastLexeme=lastLexeme[:-1]+","
                    #bestLexemes[-1]=lastLexeme
                    bestLexeme=bestLexeme.capitalize() # shorten sentences a bit
                    
         acceptedLexemes["rejected"] = acceptedLexemes["rejected"]+1  
        
   print("  expanding with: \"{}\" bestscore: {} bestphrase: {}".format(bestLexeme, embeddings[bestIndex], expansions[bestIndex]))
   bestLexemes.append(bestLexeme)

   output = dt.detokenize(bestLexemes)
   Path(outputfile).write_text(output)
    
   #stats 
   acceptedLexemes["total_score"] = acceptedLexemes["total_score"]+embeddings[bestIndex]
   print(acceptedLexemes)
   Path(statsfile).write_text(str(acceptedLexemes))


In [ ]:
outputfile =  outfile + "0" # str(ind)
statsfile = sfile + "0" # str(ind)
input = input14
    
bestLexemes = []
acceptedLexemes = {"accepted":0, "rejected":0, "total_score": 0, "candidate_percent":[]}

sentences = sent_tokenize(input)
#sentences = tokenizer.tokenize(input)

for sent in sentences:
   print("Input sentence: {}".format(sent))
   generateLexeme(sent)

output = dt.detokenize(bestLexemes)
Path(outputfile).write_text(output)

print("\n")
print(output)
print("\n")
print("------------------------------------------------------------")
print(input)

    




Input sentence: 

The Bible (from Koine Greek τὰ βιβλία, tà biblía, "the books")[a] is a collection of sacred texts or scriptures.
Leading output with seed phrase: The Bible (from Koine Greek τὰ βιβλία,
seed: "The Bible (from Koine Greek τὰ βιβλία,"
	  processing lexeme: "tà biblía, ``the books``) [a] is a collection of sacred texts or scriptures."
Original ending: .
  expanding with: ""Litana"), is a collection of texts known as the Tanakh." bestscore: 0.8998443484306335 bestphrase: "Litana"), is a collection of texts known as the Tanakh.
{'accepted': 1, 'rejected': 0, 'total_score': 0.8998443484306335, 'candidate_percent': [0.0001]}
Input sentence: Varying parts of the Bible are considered to be a product of divine inspiration and a record of the relationship between God and humans by Christians, Jews, Samaritans, and Rastafari.
seed: "The Bible (from Koine Greek τὰ βιβλία, "Litana"), is a collection of texts known as the Tanakh."
	  processing lexeme: "Varying parts of the Bible are

In [ ]:
from tensorflow.contrib.memory_stats.python.ops.memory_stats_ops import BytesInUse
with tf.device('/device:GPU:0'):  
  bytes_in_use = BytesInUse()
with tf.Session() as sess:
  print(sess.run(bytes_in_use))